# Рекомендаційні системи. Лабораторна робота №4.

## 1. Імпортуємо необхідні бібліотеки


In [42]:
from enum import unique

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

## 2. Завантаження та перегляд датасету


In [43]:
songs = pd.read_csv("./song_data.csv")
songs

,user_id,song_id,listen_count,title,artist,song
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Jack Johnson,The Cove - Jack Johnson
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Paco De Lucia,Entre Dos Aguas - Paco De Lucia
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Kanye West,Stronger - Kanye West
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,Jack Johnson,Constellations - Jack Johnson
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,Foo Fighters,Learn To Fly - Foo Fighters
...,...,...,...,...,...,...
1116604,276e43ad698705e5011e5091f367d951b21246f5,SOWTYVO12AB0180BDA,8,Surf Wax America,Weezer,Surf Wax America - Weezer
1116605,276e43ad698705e5011e5091f367d951b21246f5,SOWZHOY12A6701F9FC,1,Kill The King,Megadeth,Kill The King - Megadeth
1116606,276e43ad698705e5011e5091f367d951b21246f5,SOXJOZW12A6701E90E,7,You Think I Ain't Worth A Dollar_ But I Feel L...,Queens Of The Stone Age,You Think I Ain't Worth A Dollar_ But I Feel L...
1116607,276e43ad698705e5011e5091f367d951b21246f5,SOXNKNC12A6701F7A8,1,Superbeast,Rob Zombie,Superbeast - Rob Zombie


In [44]:
len(songs) #1116609
# Через те, що датасет занадто великий, обмежимос 10000 записами
songs = songs.head(10000)


## 2. Переглядаємо найбільш популярні пісні

In [45]:
# Групуємо пісні за назвою та кількістю користувачів, які прослухали цю пісню
songs_grouped = songs.groupby('song').agg({'listen_count': 'count'}).reset_index()
songs_grouped

,song,listen_count
0,& Down - Boys Noize,2
1,' Cello Song - Nick Drake,1
2,'97 Bonnie & Clyde - Eminem,2
3,'Round Midnight - Amy Winehouse,1
4,'Round Midnight - Miles Davis,3
...,...,...
5150,paranoid android - Christopher O'Riley,1
5151,re:stacks - Bon Iver,1
5152,sillyworld (Album Version) - Stone Sour,2
5153,sun drums and soil - Four Tet,1


In [46]:
# Підраховуємо загальну кількість прослуховувань для всіх пісень
grouped_sum = songs_grouped.listen_count.sum()
grouped_sum

np.int64(10000)

In [47]:
# Розраховуємо відсоток популярності кожної пісні
songs_grouped['percentage'] = songs_grouped.listen_count.div(grouped_sum) * 100
songs_grouped

,song,listen_count,percentage
0,& Down - Boys Noize,2,0.02
1,' Cello Song - Nick Drake,1,0.01
2,'97 Bonnie & Clyde - Eminem,2,0.02
3,'Round Midnight - Amy Winehouse,1,0.01
4,'Round Midnight - Miles Davis,3,0.03
...,...,...,...
5150,paranoid android - Christopher O'Riley,1,0.01
5151,re:stacks - Bon Iver,1,0.01
5152,sillyworld (Album Version) - Stone Sour,2,0.02
5153,sun drums and soil - Four Tet,1,0.01


In [48]:
# Сортуємо значення за спаданням процента популярності
songs_grouped.sort_values("percentage", ascending=False).head(10)

,song,listen_count,percentage
3699,Sehr kosmisch - Harmonia,50,0.50
1067,Dog Days Are Over (Radio Edit) - Florence + Th...,48,0.48
4690,Undo - Björk,46,0.46
5117,You're The One - Dwight Yoakam,43,0.43
3691,Secrets - OneRepublic,42,0.42
3511,Revelry - Kings Of Leon,38,0.38
1889,Horn Concerto No. 4 in E flat K495: II. Romanc...,36,0.36
1407,Fireflies - Charttraxx Karaoke,33,0.33
4412,The Scientist - Coldplay,28,0.28
1829,Hey_ Soul Sister - Train,25,0.25


## 3. Вивчимо кількість записів та кількість користувачів

In [49]:
len(songs) # 1116609
# Витягуємо унікальні ідентифікатори користувачів
users = songs.user_id.unique()
len(users) # 66346

595

## 4. Створюємо рекомендаційну систему для пісень - __simple popularity-based recommender__, що дозволяє визначити найбільш популярні пісні.


In [50]:
# Розділяємо датасет на тренувальну та тестову вибірки
train_data, test_data = train_test_split(songs, test_size=0.2, random_state=0)
train_data.head(5)

,user_id,song_id,listen_count,title,artist,song
7389,354cfdb566f543bb5b810a4d8959d974a30797fd,SOPLUOT12A6D4F7AC3,1,Intergalactic,Beastie Boys,Intergalactic - Beastie Boys
9275,0ec9cc33028dff6209aa49bf645ef64bdcbe00fc,SOHAZRY12A8C13BC47,12,Southside,Common / Kanye West,Southside - Common / Kanye West
2995,8fce200f3912e9608e3b1463cdb9c3529aab5c08,SOIBPQQ12A6310F0F4,1,I Bloom Blaum,Coldplay,I Bloom Blaum - Coldplay
5316,7ef2a3b074b34984f3f677bddde0f1813486cc10,SOUJWJW12A63110848,1,Sound Check (Gravity),Gorillaz,Sound Check (Gravity) - Gorillaz
356,2b6c2f33bc0e887ea7c4411f58106805a1923280,SONYEOJ12A8C142E86,6,Breed,Nirvana,Breed - Nirvana


In [51]:
# Створюємо клас popularity_recommender_py - рекомендаційну систему, яка базується на вибірці популярних даних серед всіх користувачів
class popularity_recommender_py():
    def __init__(self):
        self.train_data = None
        self.user_id = None
        self.item_id = None
        self.popularity_recommendations = None

    def create(self, train_data, user_id, item_id):
        # Зберігаємо передані аргументи у внутрішні змінні класу
        self.train_data = train_data
        self.user_id = user_id
        self.item_id = item_id
        # Групуємо дані за назвою пісні та рахуємо скільки користувачів прослухали кожну пісню
        train_data_grouped = train_data.groupby(self.item_id).agg({self.user_id: 'count'}).reset_index()
        # Перейменовуємо стовпець з кількістю прослуховувань
        train_data_grouped.rename(columns={self.user_id: 'score'}, inplace=True)
        # Спочатку сортуємо за score, потім за item_id (назвою пісні) для стабільного порядку
        train_data_sort = train_data_grouped.sort_values(
        ['score', self.item_id], ascending=[0,1])
        # Створюємо колонку "rank", яка містить місце пісні в рейтингу (якщо однакові score - найперша у ДФ отримає вищий ранг)
        train_data_sort['rank'] = train_data_sort['score'].rank(
        ascending=0, method='first')
        # Зберігаємо топ-10 найпопулярніших пісень у змінну popularity_recommendations
        self.popularity_recommendations = train_data_sort.head(10)


    def recommend(self, user_id):
        # Копіюємо рекомендовані пісні
        user_recommendations = self.popularity_recommendations
        # Додаємо колонку, щоб вказати, кому ці рекомендації адресовані
        user_recommendations['user_id'] = user_id
        # Переносимо колонку user_id в самий початок таблиці для зручності
        cols = user_recommendations.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        user_recommendations = user_recommendations[cols]
        return user_recommendations

In [52]:
# Ініціалізуємо модель
popularity_m = popularity_recommender_py()
popularity_m.create(train_data, "user_id", "song")

## 5. Використання моделі для передбачення


In [53]:
# Рекомендації для першого користувача в списку
popularity_m.recommend(songs.user_id[0])

,user_id,song,score,rank
3228,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Sehr kosmisch - Harmonia,44,1.0
929,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Dog Days Are Over (Radio Edit) - Florence + Th...,40,2.0
4097,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Undo - Björk,35,3.0
3222,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Secrets - OneRepublic,34,4.0
4464,b80344d063b5ccb3212f76538f3d9e43d87dca9e,You're The One - Dwight Yoakam,33,5.0
1231,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Fireflies - Charttraxx Karaoke,29,6.0
3072,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Revelry - Kings Of Leon,29,7.0
1654,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Horn Concerto No. 4 in E flat K495: II. Romanc...,28,8.0
3849,b80344d063b5ccb3212f76538f3d9e43d87dca9e,The Scientist - Coldplay,23,9.0
1002,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Drop The World - Lil Wayne / Eminem,22,10.0


In [54]:
# Рекомендації для другого користувача в списку
popularity_m.recommend(songs.user_id[1])

,user_id,song,score,rank
3228,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Sehr kosmisch - Harmonia,44,1.0
929,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Dog Days Are Over (Radio Edit) - Florence + Th...,40,2.0
4097,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Undo - Björk,35,3.0
3222,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Secrets - OneRepublic,34,4.0
4464,b80344d063b5ccb3212f76538f3d9e43d87dca9e,You're The One - Dwight Yoakam,33,5.0
1231,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Fireflies - Charttraxx Karaoke,29,6.0
3072,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Revelry - Kings Of Leon,29,7.0
1654,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Horn Concerto No. 4 in E flat K495: II. Romanc...,28,8.0
3849,b80344d063b5ccb3212f76538f3d9e43d87dca9e,The Scientist - Coldplay,23,9.0
1002,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Drop The World - Lil Wayne / Eminem,22,10.0


### Інтерпретація результатів
Кожному користувачу рекомендуються однакові пісні, оскільки модель базується на найбільш популярній вибірці, тому в ній не враховуються особисті вподобання користувачів.

## 6. Побудуємо рекомендаційну систему з персоналізацією, що заснована на схожості


In [55]:
# Будуємо клас item_similarity_recommender_py - рекомендаційну систему, яка базується на схожості між піснями на основі того, хто їх слухав
class item_similarity_recommender_py:
    # user_id — назва колонки з ідентифікатором користувача
    # item_id — назва колонки з об’єктом (назва пісні)
    # train_data — датафрейм з історією прослуховувань
    # cooccurence_matrix — матриця схожості між піснями
    # songs_dict — словник, що ставить у відповідність кожній пісні унікальний індекс
    # rev_songs_dict — зворотний словник: з індексу у назву пісні
    # item_similarity_recommendations — таблиця з результатами рекомендацій
    def __init__(self):
        self.train_data = None
        self.user_id = None
        self.item_id = None
        self.cooccurence_matrix = None
        self.songs_dict = None
        self.rev_songs_dict = None
        self.item_similarity_recommendations = None

    # Метод отримання унікального списку пісень, які слухав конкретний користувач
    def get_user_items(self, user):
        user_data = self.train_data[self.train_data[self.user_id] == user]
        user_items = list(user_data[self.item_id].unique())
        return user_items

    # Метод отримання унікального списку користувачів, які слухали конкретну пісню
    def get_item_users(self, item):
        item_data = self.train_data[self.train_data[self.item_id] == item]
        # Отримання унікальних значень user_id - перетворення у множину для подальшої роботи з перетинами
        item_users = set(item_data[self.user_id].unique())
        return item_users

    # Метод отримання всіх унікальних пісень з тренувального набору даних
    def get_all_items_train_data(self):
        all_items = list(self.train_data[self.item_id].unique())
        return all_items

   # Побудова матриці співзустрічей (cooccurrence matrix)
    def construct_cooccurrence_matrix(self, user_songs, all_songs):
    # Отримуємо користувачів, які слухали кожну пісню з user_songs
        user_songs_users = []
        for i in range(0, len(user_songs)):
            user_songs_users.append(self.get_item_users(user_songs[i]))

        # Ініціалізуємо матрицю співзустрічей розміру
        # (кількість user_songs) × (кількість all_songs)
        cooccurrence_matrix = np.matrix(np.zeros(shape=(len(user_songs), len(all_songs))), float)

        # Обчислюємо схожість між піснями користувача і всіма унікальними піснями з тренувальних даних
        for i in range(0, len(all_songs)):

            # Отримуємо унікальних слухачів (користувачів) для пісні i
            songs_i_data = self.train_data[self.train_data[self.item_id] == all_songs[i]]
            users_i = set(songs_i_data[self.user_id].unique())

            for j in range(0, len(user_songs)):

                # Отримуємо унікальних слухачів (користувачів) для пісні j
                users_j = user_songs_users[j]

                # Обчислюємо перетин слухачів пісень i та j
                users_intersection = users_i.intersection(users_j)

                # Обчислюємо значення [j,i] у матриці співзустрічей як індекс Жаккара
                if len(users_intersection) != 0:
                    # Обчислюємо об'єднання слухачів пісень i та j
                    users_union = users_i.union(users_j)

                    cooccurrence_matrix[j, i] = float(len(users_intersection)) / float(len(users_union))
                else:
                    cooccurrence_matrix[j, i] = 0

        return cooccurrence_matrix

    # Використовуємо матрицю співзустрічей для формування топ-рекомендацій
    def generate_top_recommendations(self, user, cooccurrence_matrix, all_songs, user_songs):

        # Виводимо кількість ненульових значень у матриці співзустрічей
        print("Кількість ненульових значень у матриці співзустрічей: %d" % np.count_nonzero(cooccurrence_matrix))

        # Обчислюємо середнє зважене значення оцінок для всіх пісень користувача
        user_sim_scores = cooccurrence_matrix.sum(axis=0) / float(cooccurrence_matrix.shape[0])
        user_sim_scores = np.array(user_sim_scores)[0].tolist()

        # Сортуємо індекси пісень за оцінками схожості у спадаючому порядку
        # Зберігаємо і значення, і індекс
        sort_index = sorted(((e, i) for i, e in enumerate(list(user_sim_scores))), reverse=True)

        # Створюємо датафрейм для збереження рекомендацій
        columns = ['user_id', 'song', 'score', 'rank']
        df = pd.DataFrame(columns=columns)

        # Заповнюємо датафрейм топ-10 рекомендаціями на основі схожості
        rank = 1
        for i in range(0, len(sort_index)):
            # Якщо значення не NaN і пісня ще не прослухана користувачем — додаємо до рекомендацій
            if ~np.isnan(sort_index[i][0]) and all_songs[sort_index[i][1]] not in user_songs and rank <= 10:
                df.loc[len(df)] = [user, all_songs[sort_index[i][1]], sort_index[i][0], rank]
                rank += 1

        # Обробляємо ситуацію, коли немає жодної рекомендації
        if df.shape[0] == 0:
            print("У поточного користувача немає пісень для навчання моделі рекомендацій на основі схожості.")
            return -1
        else:
            return df

    def create(self, train_data, user_id, item_id):
        # Зберігаємо тренувальні дані та назви стовпців для користувачів і пісень
        self.train_data = train_data
        self.user_id = user_id
        self.item_id = item_id

    def recommend(self, user):
        # Отримуємо всі унікальні пісні, які слухав цей користувач
        user_songs = self.get_user_items(user)
        print("Кількість унікальних пісень для користувача: %d" % len(user_songs))

        # Отримуємо всі унікальні пісні в тренувальному наборі
        all_songs = self.get_all_items_train_data()
        print("Кількість унікальних пісень у тренувальному наборі: %d" % len(all_songs))

        # Створюємо матрицю співзустрічей (cooccurrence matrix)
        cooccurrence_matrix = self.construct_cooccurrence_matrix(user_songs, all_songs)

        # Генеруємо рекомендації на основі матриці співзустрічей
        df_recommendations = self.generate_top_recommendations(user, cooccurrence_matrix, all_songs, user_songs)

        return df_recommendations

    def get_similar_items(self, item_list):
        # Задаємо список пісень, до яких шукаємо подібні
        user_songs = item_list

        # Отримуємо всі унікальні пісні з тренувального набору
        all_songs = self.get_all_items_train_data()
        print("Кількість унікальних пісень у тренувальному наборі: %d" % len(all_songs))

        # Створюємо матрицю співзустрічей для обраних пісень і всіх пісень
        cooccurence_matrix = self.construct_cooccurrence_matrix(user_songs, all_songs)

        # Порожній рядок, оскільки generate_top_recommendations вимагає аргумент user
        user = ""

        # Генеруємо рекомендації на основі матриці співзустрічей
        df_recommendations = self.generate_top_recommendations(user, cooccurence_matrix, all_songs, user_songs)

        return df_recommendations

In [56]:
# Ініціалізуємо модель
personalized_model = item_similarity_recommender_py()
personalized_model.create(train_data, "user_id", "song")

In [57]:
# Знаходимо рекомендації для конкретного користувача
current_user_id = users[5]
user_items = personalized_model.get_user_items(current_user_id)
personalized_model.recommend(current_user_id)

Кількість унікальних пісень для користувача: 13
Кількість унікальних пісень у тренувальному наборі: 4496
Кількість ненульових значень у матриці співзустрічей: 959


,user_id,song,score,rank
0,4bd88bfb25263a75bbdd467e74018f4ae570e5df,You Found Me (Album Version) - The Fray,0.058974,1
1,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Hellbound - J-Black & Masta Ace,0.058974,2
2,4bd88bfb25263a75bbdd467e74018f4ae570e5df,As The World Turns - Eminem,0.053780,3
3,4bd88bfb25263a75bbdd467e74018f4ae570e5df,I'm Back - Eminem,0.052015,4
4,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Armed And Ready (2009 Digital Remaster) - The ...,0.044322,5
5,4bd88bfb25263a75bbdd467e74018f4ae570e5df,I'm The One Who Understands (Edit Version) - War,0.044322,6
6,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Falling - Iration,0.044322,7
7,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Superman - Eminem / Dina Rae,0.039777,8
8,4bd88bfb25263a75bbdd467e74018f4ae570e5df,The Seed (2.0) - The Roots / Cody Chestnutt,0.038586,9
9,4bd88bfb25263a75bbdd467e74018f4ae570e5df,The Way You Move - OutKast,0.038586,10


### Інтерпретація результатів


При використанні моделі, яка базується на cooccurrence matrix алгоритмі, рекомендації набувають персоналізації.

## 7. Знаходимо пісні, що близькі до певної пісні


In [58]:
personalized_model.get_similar_items(['Chan Chan (Live) - Buena Vista Social Club'])

Кількість унікальних пісень у тренувальному наборі: 4496
Кількість ненульових значень у матриці співзустрічей: 22


,user_id,song,score,rank
0,,Always Like This - Bombay Bicycle Club,0.500000,1
1,,Kong - Bonobo,0.500000,2
2,,I'll Try Anything Once - The Strokes,0.500000,3
3,,Cotton Fields (The Cotton Song) (Digitally Rem...,0.500000,4
4,,You Only Live Once - The Strokes,0.500000,5
5,,The Sound of Settling (Album Version) - Death ...,0.500000,6
6,,Novocaine For The Soul - Eels,0.333333,7
7,,Every Lasting Light - The Black Keys,0.333333,8
8,,This Charming Man - The Smiths,0.333333,9
9,,Holland_ 1945 - Neutral Milk Hotel,0.333333,10


## Домашнє завдання

1. Визначити, скільки користувачів (унікальних, тобто не враховуються рядки, що
повторюються) слухали пісні конкретного виконавця по варіанту (__#5. Jack Johnson__)


In [59]:
# Фільтруємо всі рядки, де виконавець - Jack Johnson
jack_johnson_data = songs[songs.artist == "Jack Johnson"]
# Визначаємо унікальних користувачів, які слухали його пісні
unique_users = jack_johnson_data['user_id'].nunique()
print("Кількість унікальних користувачів:", unique_users)

Кількість унікальних користувачів: 21


2. Визначити n-ого по популярності співака з початку і (n = 7) 7-ого по популярності співака з кінця.


In [60]:
# Групуємо за артистами й рахуємо кількість прослуховувань
artist_counts = train_data.groupby('artist').agg({"user_id": "count"}).reset_index()

# Перейменовуємо колонку для зручності
artist_counts.columns = ['artist', 'listen_count']

# Сортуємо за кількістю прослуховувань (від найпопулярнішого до найменшого)
artist_sorted = artist_counts.sort_values(by='listen_count', ascending=False).reset_index(drop=True)

# 7-й по популярності (тобто на позиції 6, бо індексація з 0)
most_popular = artist_sorted.iloc[6]

# 7-й з кінця
least_popular = artist_sorted.iloc[-7]

print(f"7-й по популярності: {most_popular['artist']} ({most_popular['listen_count']} прослуховувань)")
print(f"7-й з кінця: {least_popular['artist']} ({least_popular['listen_count']} прослуховувань)")


7-й по популярності: Beyoncé (52 прослуховувань)
7-й з кінця: Gunther & the Sunshine Girls (1 прослуховувань)


3. Для перших 3 користувачів визначити першу пісню, що рекомендується.
Для цього виконати наступне: змінити персоналізовану модель, спадкуватися від
класу персоналізованої моделі item_similarity_recommender_py і написати метод, який
буде рекомендувати одну пісню (аналог generate top recommendations).

Можна використати вбудовані в клас item_similarity_recommender_py методи для:
* генерації матриці construct_cooccurence_matrix
* повернення всіх пісень get_all_items_train_data
* повернення всіх пісень для користувача по Id користувача get_user_items

In [61]:
# Cтворюємо модель яка успадковує item_similarity_recommender_py
class PersonalItemRecommender(item_similarity_recommender_py):
    def recommend_one_song(self, user):
        # Отримуємо список пісень, які слухав користувач
        user_songs = self.get_user_items(user)

        # Отримуємо всі унікальні пісні в датасеті
        all_songs = self.get_all_items_train_data()

        # Будуємо матрицю співзустрічей
        cooccurence_matrix = self.construct_cooccurrence_matrix(user_songs, all_songs)

        # Обчислюємо середній скор кожної пісні на основі співзустрічей
        user_sim_scores = cooccurence_matrix.sum(axis=0) / float(cooccurence_matrix.shape[0])
        user_sim_scores = np.array(user_sim_scores)[0].tolist()

        # Сортуємо пісні за скором у порядку спадання
        sort_index = sorted(((score, idx) for idx, score in enumerate(user_sim_scores)), reverse=True)

        # Обираємо першу пісню, яку ще не слухав користувач
        for score, idx in sort_index:
            if not np.isnan(score) and all_songs[idx] not in user_songs:
                return all_songs[idx]  # Рекомендована пісня

        # Якщо немає нових пісень
        return None

In [63]:
# Ініціалізуємо модель
model = PersonalItemRecommender()
model.create(train_data, 'user_id', 'song')
# Отримуємо перші 3 унікальні user_id
users = songs['user_id'].unique()[:3]
# Для кожного користувача — одна рекомендація
for user in users:
    song = model.recommend_one_song(user)
    print(f'Користувач {user}: рекомендована пісня — {song}')

Користувач b80344d063b5ccb3212f76538f3d9e43d87dca9e: рекомендована пісня — Your Protector - Fleet Foxes
Користувач 85c1f87fea955d09b4bec2e36aee110927aedf9a: рекомендована пісня — Puppets - Atmosphere
Користувач bd4c6e843f00bd476847fb75c47b4fb430a06856: рекомендована пісня — Dead Souls [Re-mastered] - Joy Division
